In [1]:
!pip install --disable-pip-version-check --quiet -U langchain==0.2.16
!pip install --disable-pip-version-check --quiet -U langchain_openai==0.1.23
!pip install --disable-pip-version-check --quiet -U langgraph==0.2.19
!pip install --disable-pip-version-check --quiet -U langchainhub==0.1.21
!pip install --disable-pip-version-check --quiet -U tavily-python==0.4.0
!pip install --disable-pip-version-check --quiet -U langchain-community==0.2.16
!pip install --disable-pip-version-check --quiet -U python-dotenv==1.0.1

In [1]:
#Load the keys

from dotenv import load_dotenv
import os

load_dotenv('config.env')
assert os.getenv('OPENAI_API_KEY') is not None
assert os.getenv('TAVILY_API_KEY') is not None
print("✅ Keys loaded successfully!")

✅ Keys loaded successfully!


In [2]:
#Imports + Helpers + State

import time
from langgraph.graph import MessagesState, START, StateGraph, END
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig
from langchain_community.tools.tavily_search import TavilySearchResults
from IPython.display import Image, display
import os

# Helper functions (FROM ORIGINAL STARTER)
def display_text_to_user(text):
    print(text)
    time.sleep(1)

def ask_user_for_input(input_description):
    response = input(input_description)
    return response

# State
# class State(MessagesState):
#     topic: str = ""
#     search_results: str = ""
#     summary: str = ""
#     quiz_question: str = ""
#     user_answer: str = ""
#     grade: str = ""
#     continue_session: bool = True
        
#modified state class as per Rubric suggestions
class State(MessagesState):
    # Existing fields
    topic: str = ""
    search_results: str = ""
    summary: str = ""
    quiz_question: str = ""
    user_answer: str = ""
    grade: str = ""
    continue_session: bool = True
    
    # NEW: Stand-out feature fields
    difficulty: str = "medium"        # easy/medium/hard
    num_questions: int = 1            # how many quiz questions
    current_question_num: int = 0     # track current question
    
    correct_answers: int = 0
    quiz_questions: list = []         # store multiple questions
    related_subjects: list = []       # suggested related topics

# Model and Tools
#model = ChatOpenAI(temperature=0, streaming=True)
model = ChatOpenAI(
    temperature=0,
    streaming=True,
    openai_api_base="https://openai.vocareum.com/v1",  # Udacity proxy
    openai_api_key=os.getenv('OPENAI_API_KEY')
)


search_tool = TavilySearchResults(max_results=5)

print("✅ Imports and State ready!")

✅ Imports and State ready!


In [4]:
#Node Functions
# Node 1
def ask_topic(state: State):
    """Node 1: Ask patient what health topic they want to learn about."""
    display_text_to_user("👋 Welcome to HealthBot!")
    display_text_to_user("I can help you learn about health topics and medical conditions.")
    topic = ask_user_for_input("What health topic would you like to learn about? ")
    return {"topic": topic}

#new node added to check the difficultie level 
def ask_difficulty(state: State):
    """NEW Node: Ask patient for difficulty level."""
    display_text_to_user("\n📊 Choose your learning difficulty:")
    display_text_to_user("  Easy   - Simple overview, basic questions")
    display_text_to_user("  Medium - Detailed summary, moderate questions")
    display_text_to_user("  Hard   - In-depth summary, complex questions")
    
    while True:
        difficulty = ask_user_for_input(
            "\nEnter difficulty (easy/medium/hard): "
        ).strip().lower()
        
        if difficulty in ['easy', 'medium', 'hard']:
            break
        display_text_to_user("⚠️ Please enter: easy, medium, or hard")
    
    display_text_to_user(f"✅ Difficulty set to: {difficulty.upper()}")
    return {"difficulty": difficulty}


# Node 2
def search_health_info(state: State):
    """Node 2: Search Tavily for relevant medical information."""
    display_text_to_user(f"\n🔍 Searching for: {state['topic']}...")
    search_query = f"{state['topic']} medical information symptoms treatment"
    results = search_tool.invoke(search_query)
    combined_results = "\n\n".join([
        f"Source: {r.get('url', 'N/A')}\n{r.get('content', '')}"
        for r in results
    ])
    display_text_to_user("✅ Found relevant medical information!")
    return {"search_results": combined_results}

# Node 3
def summarize_info(state: State):
    """Node 3: Summarize based on difficulty level."""
    display_text_to_user("\n📝 Creating a patient-friendly summary...")
    
    # Difficulty-specific instructions
    difficulty_prompts = {
        "easy": """
            - Use very simple language (5th grade reading level)
            - Keep it SHORT (2-3 paragraphs)
            - Focus only on the most important basics
            - Avoid medical jargon completely
            - Use bullet points for symptoms and treatments
        """,
        "medium": """
            - Use clear, simple language
            - Keep it MODERATE (3-4 paragraphs)
            - Cover what it is, symptoms, treatment, prevention
            - Minimal medical jargon (explain if used)
        """,
        "hard": """
            - Use detailed medical language (explain complex terms)
            - Make it COMPREHENSIVE (5-6 paragraphs)
            - Include pathophysiology, complications, detailed treatments
            - Include statistics and research findings where available
            - Cover edge cases and nuances
        """
    }
    
    difficulty = state.get('difficulty', 'medium')
    difficulty_instruction = difficulty_prompts.get(difficulty, difficulty_prompts['medium'])
    
    prompt = f"""You are a medical educator creating a {difficulty.upper()} level summary.

    Topic: {state['topic']}

    Instructions for {difficulty.upper()} level:
    {difficulty_instruction}

    Medical Information to summarize:
    {state['search_results']}

    Patient-friendly {difficulty.upper()} level summary:"""

    response = model.invoke(prompt)
    summary = response.content
    return {"summary": summary}



# Node 4
def present_summary(state: State):
    """Node 4: Present the summary to the patient."""
    display_text_to_user("\n" + "="*60)
    display_text_to_user("📋 HEALTH INFORMATION SUMMARY")
    display_text_to_user("="*60)
    display_text_to_user(state['summary'])
    display_text_to_user("="*60)
    return state

# Node 5
def wait_for_ready(state: State):
    """Node 5: Wait for patient to indicate readiness for quiz."""
    ask_user_for_input("\n✅ Type 'ready' when you want to take the comprehension check: ")
    return state

#New node added to Ask Number of Questions

def ask_num_questions(state: State):
    """NEW Node: Ask how many quiz questions patient wants."""
    display_text_to_user("\n❓ How many quiz questions would you like?")
    
    while True:
        try:
            num = int(ask_user_for_input("Enter number (1-5): ").strip())
            if 1 <= num <= 5:
                break
            display_text_to_user("⚠️ Please enter a number between 1 and 5")
        except ValueError:
            display_text_to_user("⚠️ Please enter a valid number")
    
    display_text_to_user(f"✅ Will generate {num} question(s)")
    return {
        "num_questions": num,
        "current_question_num": 0,
        "quiz_questions": []
    }


# Node 6
def generate_quiz(state: State):
    current_num = state.get('current_question_num', 0) + 1
    total = state.get('num_questions', 1)
    difficulty = state.get('difficulty', 'medium')
    existing = state.get('quiz_questions', [])
    
    display_text_to_user(f"\n🧠 Generating question {current_num} of {total}...")
    
    difficulty_styles = {
        "easy": "Simple recall question.",
        "medium": "Understanding and interpretation required.",
        "hard": "Complex critical thinking required."
    }
    
    # Build strong avoid-duplicate instruction
    avoid_text = ""
    if existing:
        avoid_text = f"""
        IMPORTANT - These topics were ALREADY asked. Ask about DIFFERENT aspects:
        {chr(10).join([f"- Already asked: {q.split(chr(10))[0][:80]}" for q in existing])}

        You MUST ask about a COMPLETELY DIFFERENT aspect of {state['topic']}.
        """
    
    prompt = f"""Create quiz question {current_num} of {total} about {state['topic']}.

    Difficulty: {difficulty.upper()} - {difficulty_styles.get(difficulty)}

    {avoid_text}

    Summary to base question on:
    {state['summary']}

    Rules:
    - Question MUST be about a DIFFERENT aspect than previous questions
    - For question 1: Ask about WHAT it is / definition
    - For question 2: Ask about SYMPTOMS or causes  
    - For question 3: Ask about TREATMENT or prevention
    - For question 4+: Ask about complications or statistics

    Format EXACTLY:
    Question: [question text]
    A) [option]
    B) [option]
    C) [option]
    D) [option]
    Correct Answer: [A/B/C/D]"""

    response = model.invoke(prompt)
    new_question = response.content
    updated_questions = existing + [new_question]
    
    return {
        "quiz_question": new_question,
        "quiz_questions": updated_questions,
        "current_question_num": current_num
    }


"""
Issue 2: Score Display Confusing**

After Q1 wrong: "Score: 0/2 questions completed" ← confusing
After Q2 wrong: "Score: 1/2 questions completed" ← should be 2/2
"""
# Node 7
def present_quiz(state: State):
    """Node 7: Present the quiz question to the patient."""
    question_only = state['quiz_question'].split("Correct Answer:")[0].strip()
    display_text_to_user("\n" + "="*60)
    display_text_to_user("❓ COMPREHENSION CHECK")
    display_text_to_user("="*60)
    display_text_to_user(question_only)
    display_text_to_user("="*60)
    return state

# Node 8
def get_answer(state: State):
    """Node 8: Get the patient's answer."""
    user_answer = ask_user_for_input("\nYour answer (A/B/C/D): ")
    return {"user_answer": user_answer.strip().upper()}

# Node 9
def grade_answer(state: State):
    display_text_to_user("\n📊 Evaluating your answer...")
    
    difficulty = state.get('difficulty', 'medium')
    current = state.get('current_question_num', 1)
    total = state.get('num_questions', 1)
    correct_so_far = state.get('correct_answers', 0)
    
    feedback_styles = {
        "easy": "Simple encouraging feedback.",
        "medium": "Balanced feedback with explanation.",
        "hard": "Detailed technical feedback."
    }
    
    prompt = f"""Grade this answer. Be precise about correct/incorrect.

    Quiz Question:
    {state['quiz_question']}

    Patient's Answer: {state['user_answer']}

    Summary (citations only from here):
    {state['summary']}

    Format EXACTLY:
    Grade: [Correct/Incorrect]
    Explanation: [why right/wrong]
    Citation: "[quote from summary]"
    Encouragement: [motivating message]"""

    response = model.invoke(prompt)
    grade_text = response.content
    
    # Track correct answers
    is_correct = "grade: correct" in grade_text.lower()
    new_correct = correct_so_far + (1 if is_correct else 0)
    
    # Add score to display
    score_line = f"\n📊 Progress: {current}/{total} questions done | ✅ {new_correct} correct"
    
    return {
        "grade": grade_text + score_line,
        "correct_answers": new_correct
    }
# Node 10
def present_grade(state: State):
    """Node 10: Present the grade and explanation."""
    display_text_to_user("\n" + "="*60)
    display_text_to_user("📈 QUIZ RESULTS")
    display_text_to_user("="*60)
    display_text_to_user(state['grade'])
    display_text_to_user("="*60)
    return state


#New Node added 

def suggest_related_subjects(state: State):
    """NEW Node: Show final score THEN suggest related topics."""
    
    
    correct = state.get('correct_answers', 0)
    total = state.get('num_questions', 1)
    percentage = int((correct / total) * 100) if total > 0 else 0
    
    display_text_to_user("\n" + "="*60)
    display_text_to_user("🏆 SESSION COMPLETE!")
    display_text_to_user("="*60)
    display_text_to_user(f"📚 Topic:      {state['topic'].upper()}")
    display_text_to_user(f"🎯 Difficulty: {state.get('difficulty', 'medium').upper()}")
    display_text_to_user(f"✅ Score:      {correct}/{total} correct ({percentage}%)")
    
    # Performance message
    if percentage == 100:
        display_text_to_user("⭐ PERFECT SCORE! Outstanding!")
    elif percentage >= 70:
        display_text_to_user("👍 Great job! Keep it up!")
    elif percentage >= 50:
        display_text_to_user("📖 Good effort! Review the summary.")
    else:
        display_text_to_user("💪 Keep studying - you'll improve!")
    
    display_text_to_user("="*60)
    
    display_text_to_user("\n🔗 Finding related topics you might like...")
    
    prompt = f"""Based on the health topic "{state['topic']}", suggest exactly 3 related 
        health topics that would be educational and interesting to learn about next.

        Format EXACTLY:
        1. [topic name]: [one sentence why it's related]
        2. [topic name]: [one sentence why it's related]  
        3. [topic name]: [one sentence why it's related]

        Related topics:"""

    response = model.invoke(prompt)
    suggestions_text = response.content
    
    lines = [l.strip() for l in suggestions_text.strip().split('\n') if l.strip()]
    related = [l for l in lines if l and l[0] in ['1', '2', '3']]
    
    display_text_to_user("\n" + "="*60)
    display_text_to_user("💡 RELATED TOPICS YOU MIGHT LIKE:")
    display_text_to_user("="*60)
    for topic in related:
        display_text_to_user(topic)
    display_text_to_user("="*60)
    
    return {"related_subjects": related}


# Node 11
def ask_continue(state: State):
    """Node 11: Ask if patient wants another topic with suggestions."""
    related = state.get('related_subjects', [])
    
    display_text_to_user("\n🔄 What would you like to do next?")
    display_text_to_user("  'yes'  - Learn about a NEW topic (you choose)")
    
    # Show numbered related topics
    for i, topic in enumerate(related[:3], 1):
        topic_name = topic.split(':')[0].replace(str(i)+'.', '').strip()
        display_text_to_user(f"  '{i}'   - Learn about: {topic_name}")
    
    display_text_to_user("  'no'   - Exit HealthBot")
    
    response = ask_user_for_input("\nYour choice: ").strip().lower()
    
    # Handle choices
    if response in ['1', '2', '3'] and related:
        # User chose a related topic
        try:
            idx = int(response) - 1
            chosen = related[idx].split(':')[0].replace(response+'.', '').strip()
            display_text_to_user(f"✅ Great choice! Loading: {chosen}")
            return {
                "continue_session": True,
                "topic": chosen  # Pre-fill the topic!
            }
        except (IndexError, ValueError):
            pass
    
    continue_session = response in ['yes', 'y']
    return {"continue_session": continue_session}

# Node 12a
def reset_state(state: State):
    kept_topic = state.get('topic', '')
    return {
        "topic": kept_topic,
        "search_results": "",
        "summary": "",
        "quiz_question": "",
        "quiz_questions": [],
        "user_answer": "",
        "grade": "",
        "continue_session": True,
        "current_question_num": 0,
        "num_questions": 1,
        "related_subjects": [],
        "correct_answers": 0,  # ← Reset score
        "difficulty": "medium",
        "messages": []
    }
# Node 12b
def end_session(state: State):
    """Node 12b: End the session."""
    display_text_to_user("\n👋 Thank you for using HealthBot!")
    display_text_to_user("Stay healthy and informed! 💙")
    return {"continue_session": False}

# Router
def should_continue(state: State):
    """Router: Continue or end?"""
    if state.get('continue_session', True):
        return "continue"
    else:
        return "end"

print("✅ All 12 nodes and extra nodes for advanced complexity features are  defined!")

✅ All 12 nodes and extra nodes for advanced complexity features are  defined!


In [5]:
def check_more_questions(state: State):
    """Router node: Check if more questions needed."""
    return state  # Just passes state through


def should_ask_more_questions(state: State):
    """Router: More questions or done?"""
    current = state.get('current_question_num', 0)
    total = state.get('num_questions', 1)
    
    if current < total:
        return "more_questions"
    else:
        return "done"


def should_continue(state: State):
    """Router: Continue or end?"""
    if state.get('continue_session', True):
        return "continue"
    else:
        return "end"


In [6]:
workflow = StateGraph(State)

# Add all nodes
workflow.add_node("ask_topic", ask_topic)
workflow.add_node("ask_difficulty", ask_difficulty)          # NEW
workflow.add_node("search", search_health_info)
workflow.add_node("summarize", summarize_info)
workflow.add_node("present_summary", present_summary)
workflow.add_node("wait_ready", wait_for_ready)
workflow.add_node("ask_num_questions", ask_num_questions)   # NEW
workflow.add_node("generate_quiz", generate_quiz)
workflow.add_node("present_quiz", present_quiz)
workflow.add_node("get_answer", get_answer)
workflow.add_node("grade_node", grade_answer)
workflow.add_node("present_grade", present_grade)
workflow.add_node("suggest_related", suggest_related_subjects)  # NEW
workflow.add_node("ask_continue", ask_continue)
workflow.add_node("reset", reset_state)
workflow.add_node("end", end_session)

# Define edges
workflow.add_edge(START, "ask_topic")
workflow.add_edge("ask_topic", "ask_difficulty")            # NEW
workflow.add_edge("ask_difficulty", "search")               # NEW
workflow.add_edge("search", "summarize")
workflow.add_edge("summarize", "present_summary")
workflow.add_edge("present_summary", "wait_ready")
workflow.add_edge("wait_ready", "ask_num_questions")        # NEW
workflow.add_edge("ask_num_questions", "generate_quiz")     # NEW
workflow.add_edge("generate_quiz", "present_quiz")
workflow.add_edge("present_quiz", "get_answer")
workflow.add_edge("get_answer", "grade_node")
workflow.add_edge("grade_node", "present_grade")
workflow.add_edge("present_grade", "check_more_questions")  # NEW router

# NEW: Router for multiple questions
workflow.add_node("check_more_questions", check_more_questions)

workflow.add_conditional_edges(
    "check_more_questions",
    should_ask_more_questions,
    {
        "more_questions": "generate_quiz",    # Loop for more questions
        "done": "suggest_related"             # Move to suggestions
    }
)

workflow.add_edge("suggest_related", "ask_continue")        # NEW

# Conditional routing for continue/end
workflow.add_conditional_edges(
    "ask_continue",
    should_continue,
    {
        "continue": "reset",
        "end": "end"
    }
)

workflow.add_edge("reset", "ask_topic")
workflow.add_edge("end", END)

print("Workflow steps added and executed successfully")

Workflow steps added and executed successfully


In [7]:
#compile + display diagram 


# Compile
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Display diagram (handle gracefully if fails)
try:
    display(Image(app.get_graph().draw_mermaid_png()))
    print("✅ Visual diagram displayed!")
except Exception as e:
    print("⚠️ Visual diagram unavailable")
    print("\n📊 Text Workflow:")
    print(app.get_graph().draw_mermaid())

⚠️ Visual diagram unavailable

📊 Text Workflow:
%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	ask_topic(ask_topic)
	ask_difficulty(ask_difficulty)
	search(search)
	summarize(summarize)
	present_summary(present_summary)
	wait_ready(wait_ready)
	ask_num_questions(ask_num_questions)
	generate_quiz(generate_quiz)
	present_quiz(present_quiz)
	get_answer(get_answer)
	grade_node(grade_node)
	present_grade(present_grade)
	suggest_related(suggest_related)
	ask_continue(ask_continue)
	reset(reset)
	end(end)
	check_more_questions(check_more_questions)
	__end__([<p>__end__</p>]):::last
	__start__ --> ask_topic;
	ask_difficulty --> search;
	ask_num_questions --> generate_quiz;
	ask_topic --> ask_difficulty;
	end --> __end__;
	generate_quiz --> present_quiz;
	get_answer --> grade_node;
	grade_node --> present_grade;
	present_grade --> check_more_questions;
	present_quiz --> get_answer;
	present_summary --> wait_ready;
	reset --> ask_topic;
	search -

In [8]:
# Run HealthBot
config = RunnableConfig(
    recursion_limit=2000,
    configurable={"thread_id": "healthbot_session_1"}
)

initial_state = {
    "messages": [],
    "topic": "",
    "search_results": "",
    "summary": "",
    "quiz_question": "",
    "quiz_questions": [],
    "user_answer": "",
    "grade": "",
    "continue_session": True,
    "difficulty": "medium",
    "num_questions": 1,
    "current_question_num": 0,
    "related_subjects": [],
    "correct_answers": 0        # ← ADD THIS LINE
}

print("🚀 Starting HealthBot...")
app.invoke(initial_state, config)

🚀 Starting HealthBot...
👋 Welcome to HealthBot!
I can help you learn about health topics and medical conditions.
What health topic would you like to learn about? fever

📊 Choose your learning difficulty:
  Easy   - Simple overview, basic questions
  Medium - Detailed summary, moderate questions
  Hard   - In-depth summary, complex questions

Enter difficulty (easy/medium/hard): easy
✅ Difficulty set to: EASY

🔍 Searching for: fever...
✅ Found relevant medical information!

📝 Creating a patient-friendly summary...

📋 HEALTH INFORMATION SUMMARY
A fever is when your body temperature is higher than normal. You may feel hot, sweaty, and shivery. Other symptoms can include fatigue, headache, and muscle aches. Fevers are usually caused by infections.

When to worry about a fever:
- Call your doctor if your temperature is 103°F or higher
- Seek immediate medical attention if you have a severe headache, rash, stiff neck, confusion, or difficulty breathing
- For babies under 3 months, call the d

{'messages': [],
 'topic': 'fever',
 'search_results': "Source: https://www.medicalnewstoday.com/articles/168266\n## When should I worry about a fever?\n\nAny adult with a temperature over 105.8°F (41°C) should get emergency medical attention. This is known as hyperpyrexia, which can have serious consequences if left untreated.\n\nPeople should also seek immediate medical treatment if a person has a fever of any temperature along with:\n\n sunburn\n chest pain\n rapid breathing\n difficulty breathing\n coughing up pus or blood\n a blue, gray, or white tint to the lips or fingers\n a severe headache\n an aversion to light\n a severe or worsening rash\n a stiff neck\n confusion\n drowsiness\n loss of consciousness\n signs of severe dehydration, such as a lack of sweat, saliva, or urine\n seizures or convulsions\n\nAdditional signs that a child needs immediate medical attention include: [...] However, it is important to take steps to prevent the spread of the infection to protect people w